<a href="https://colab.research.google.com/github/Prajwal-M-Rao/LangChain-Agent/blob/main/Other_Models_with_some_Modifications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain langchain-community langchain-openai ollama wikipedia

In [ ]:
import os
from langchain_community.llms import HuggingFacePipeline
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from transformers import pipeline

# Set up a local pipeline using a small model that can run on CPU
# This doesn't require API keys or external services
llm_pipeline = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",  # A small model that can run on CPU
    max_new_tokens=150,
    temperature=0.7,
)

# Create a LangChain wrapper around the pipeline
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Set up the Wikipedia tool
wikipedia = WikipediaAPIWrapper(
    top_k_results=2,
    doc_content_chars_max=4000
)
wikipedia_tool = WikipediaQueryRun(api_wrapper=wikipedia)

# Define the prompt template for our agent
prompt_template = """You are a helpful assistant that answers user questions using your knowledge and Wikipedia when necessary.
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original user question

Begin!

Question: {input}
{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(prompt_template)

# Create the agent
agent = create_react_agent(
    llm=llm,
    tools=[wikipedia_tool],
    prompt=prompt
)

# Create the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[wikipedia_tool],
    verbose=True,
    handle_parsing_errors=True
)

def process_query(user_query):
    """Process a natural language query through the agent."""
    try:
        result = agent_executor.invoke({"input": user_query})
        return result["output"]
    except Exception as e:
        return f"Error processing your query: {str(e)}"

# Example usage
if __name__ == "__main__":
    print("Welcome to the LangChain Wikipedia Agent!")
    print("Ask any question, and I'll use my knowledge and Wikipedia to answer.")
    print("Type 'exit' to quit.\n")

    while True:
        user_input = input("\nYour question: ")
        if user_input.lower() == "exit":
            print("Goodbye!")
            break

        print("\nProcessing your query...")
        response = process_query(user_input)
        print(f"\nResponse: {response}")

In [ ]:
!pip -q install langchain openai tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.2 MB/s eta 0:00:00


In [3]:
# import os

# os.environ["OPENAI_API_KEY"] = ""

In [2]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "" #Please add your API TOken as if the GIT wont allow to upload secret keys

In [ ]:
!pip install langchain langchain-community

In [ ]:
!pip show langchain

Name: langchain
Version: 0.3.20
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: 


In [ ]:
!pip -q install wikipedia

  Preparing metadata (setup.py) ... done


In [ ]:
from langchain.utilities import WikipediaAPIWrapper

In [ ]:
wikipedia = WikipediaAPIWrapper()

In [ ]:
wikipedia.run('Langchain')

'Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: Retrieval-augmented generation\nSummary: Retrieval-augmented generation (RAG) is a technique that enables generative artificial intelligence (Gen AI) models to retrieve and incorporate new information. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to supplement information from its pre-existing training data. This allows LLMs to use domain-specific and/or updated information.  Use cases include providing chatbot access to internal company data or generating responses based on authoritative sources.\nRAG impro

In [ ]:
pip install -U langchain-openai


In [ ]:
from langchain import OpenAI


In [ ]:
from langchain_openai import OpenAI


In [ ]:
llm = OpenAI(temperature=0)

In [ ]:
from langchain.agents import Tool
wikipedia_tool = Tool(
    name='wikipedia',
    func= wikipedia.run,
    description="Useful for when you need to look up a topic, country or person on wikipedia"
)

In [ ]:
from langchain.agents import initialize_agent

# Ensure `tools` is a list
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=[wikipedia_tool],  # Wrapped in a list
    llm=llm,
    verbose=True,
    max_iterations=3,
)

In [ ]:
zero_shot_agent.run("When was Barak Obama born?")

In [ ]:
pip install transformers langchain huggingface_hub


In [ ]:
import os
from langchain.utilities import WikipediaAPIWrapper
from langchain import HuggingFaceHub
from langchain.agents import Tool, initialize_agent

# Set your Hugging Face API key as an environment variable
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""  # Replace with your actual API key

# Initialize Wikipedia API Wrapper
wikipedia = WikipediaAPIWrapper()

# Define a tool for Wikipedia search
wikipedia_tool = Tool(
    name='wikipedia',
    func=wikipedia.run,
    description="Useful for looking up information about a topic, country, or person on Wikipedia"
)

# Load a model from Hugging Face Hub (e.g., flan-t5-xl)
# Explicitly specify the 'google/flan-t5-xl' model for text generation
llm = HuggingFaceHub(
    repo_id="google/flan-t5-xl",  # Choose a model (change as needed)
    model_kwargs={"temperature": 0.7, "max_length": 256},
    task="text-generation"  # Specify the task explicitly
)

# Initialize the agent
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=[wikipedia_tool],
    llm=llm,
    verbose=True,
    max_iterations=3,
)

# Run the agent with a query
response=zero_shot_agent.run("Tell me about Singapore")
print(response)

In [ ]:
import os
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain import HuggingFaceHub
from langchain.agents import initialize_agent, AgentType

# Set Hugging Face API token
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

# Initialize Wikipedia API Wrapper
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Load a model from Hugging Face Hub (flan-t5-xl)
llm = HuggingFaceHub(
    repo_id="google/flan-t5-xl",
    model_kwargs={"temperature": 0.7, "max_length": 256},
    task="text2text-generation"
)

# Initialize the agent, using AgentType.ZERO_SHOT_REACT_DESCRIPTION for single-turn interactions
zero_shot_agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Use zero-shot agent
    tools=[wikipedia],
    llm=llm,
    verbose=True,
    max_iterations=3,
)

# Run the agent
response = zero_shot_agent.run("Tell me about Singapore")
print(response)